In [15]:
import os
import ollama
from anytree import PreOrderIter
import json
from naive_baseline import generate_naive_baseline
from utils import generate_formal_explanations, make_decision
import pandas as pd

In [16]:
model_name = "llama3" # "mistral"

### Changed pipeline a bit, added preorder_traversal list to follow the order in which the agent decides what to do;
### function generate_comprehensive_explanation generates response from mistral https://ollama.com/library/mistral

In [ ]:
def preorder_traversal(root):
    """
    Performs a preorder traversal of a tree and returns a list of node names.
    
    :param root: The root node of the tree.
    :return: A list of node names in preorder traversal order.
    """
    preorder_traversal_nodes = []
    
    for node in PreOrderIter(root):
        current_node_name = node.name
        preorder_traversal_nodes.append(current_node_name)
    
    return preorder_traversal_nodes

def generate_natural_language_explanation(json_tree, naive_explanation, formal_explanations, tree_traversal_order, 
                                     norm, beliefs, goal, preferences, action_to_explain, chosen_trace=None):
    
    # Format key context elements in plain English
    norm_str = norm if norm else "None"
    beliefs_str = ", ".join(beliefs) if beliefs else "None"
    goal_str = goal if goal else "None"
    pref_description = f"{preferences[0]} with priority {preferences[1]}"
    chosen_trace_str = ", ".join(chosen_trace) if chosen_trace else "None"
    
    # prompt = f"""
    #     You are an AI assistant explaining your decision-making process to someone in a completely natural, conversational way. Your explanation should sound like you're having a normal conversation - not reading a report or following a template.

    #     ### CORE DATA (reference only)
    #     - JSON tree: {json_tree}
    #     - Action to explain: '{action_to_explain}'
    #     - My chosen path: {chosen_trace}
    #     - Initial beliefs: {beliefs}
    #     - Norms/restrictions: {norm}
    #     - Goal: {goal}
    #     - Preferences: {preferences}
    #     - Tree traversal: {tree_traversal_order}
    #     - naive_explanation: {naive_explanation}

    #     ### CRITICAL INSTRUCTIONS
    #     Write a natural language explanation based on the provided data. Your explanation must be conversational, comprehensive, and seamlessly integrate all the provided information. Avoid any artificial structure or technical language.
    # """

    prompt = f"""
    You are an AI assistant explaining your decision-making process to someone in a natural, conversational way. Your explanation should feel like you're casually chatting with a friend, using everyday language. Avoid formal or technical terms and make the explanation flow naturally.

    ### CORE DATA (reference only)
    - JSON tree: {json_tree} - the tree structure that represents the decision-making process
    - Action to explain: '{action_to_explain}' - the action that needs to be explained (can either be in the chosen path or not)
    - My chosen path: {chosen_trace_str} - the selected actions that were chosen at the end
    - Initial beliefs: {beliefs_str} - the set of beliefs that the agent starts with
    - Norms/restrictions: {norm_str} - the set of actions that are either prohibited or obligatory. P means prohibited, O means obligatory.
    - Goal: {goal_str} - the set of beliefs that must be true at the end of the execution of the trace
    - Preferences: {pref_description} - given as a list of factors and their order of importance
    - Tree traversal: {tree_traversal_order} - the order in which nodes were visited in the tree to generate formal explanations
    - naive_explanation: {naive_explanation} - the natural language explanation generated using the naive baseline approach

    ### CRITICAL INSTRUCTIONS
    - Your explanation must be conversational, concise, and grounded only in the provided data. Avoid formal or technical terms.
    - DO NOT invent any information that is not explicitly provided in the data. Stick strictly to the facts.
    - Avoid repeating the same information unnecessarily. Each part of the explanation should add something new.
    - Write as if you're explaining your thought process to a friend in a casual but clear and logical way.

    ABSOLUTELY DO NOT:
    - Use formal or technical terms (e.g., "preconditions" or "trace").
    - Repeat the same information multiple times.
    - Add any information that is not explicitly provided (e.g., assumptions about preferences or irrelevant details).
    - Use section headings, numbered lists, or bullet points.
    - Include meta-text about the explanation structure (e.g., "I will now explain...").

    Your explanation MUST include:
    1. A clear statement of whether you performed '{action_to_explain}' or not, based on the provided data.
    2. A description of the complete path you took, using natural and conversational language. Include the names of all nodes visited in the order they were visited.
    3. A natural integration of all initial beliefs, goals, norms, restrictions, and preferences, explaining how they influenced your decisions.
    4. A detailed account of each decision point, including:
    - All options you considered at each step.
    - Why some options were viable and others were not.
    - How your choices were connected to your initial beliefs, norms, and preferences.
    - If '{action_to_explain}' was not chosen, explain why, especially if norms or restrictions made it invalid.
    5. A summary of how your decisions align with your initial goals and beliefs.

    Keep your explanation conversational, concise, and grounded in the provided data.
    """

   
    
    # Send the prompt to your LLM
    messages = [
        {
            'role': 'user',
            'content': prompt
        }
    ]
    
    # response = requests.post(f"{api_base}/chat", json={"model": "mistral", "messages": messages})
    
    response = ollama.chat(model=model_name, messages=messages)
    
    try:
        return response['message']['content']
    except Exception as e:
        print(f"JSONDecodeError: {e}")
        return None

#### Integrated here llm explanations and save them into explanations/mistral

In [18]:
def generate_explanations_for_inputs(test_cases, cases_to_do):
    """
    Generate explanations for the inputs test cases
    :param test_cases: dictionary containing the test cases
    :param cases_to_do: list of test cases to actually generate explanations for (for easy re-running)
    :return: dictionary containing the explanations for each test case
    """
    
    # in .ipynb current directory is extracted like this
    current_dir = os.getcwd()

    # current_dir = os.path.dirname(__file__)

    baseline_explanations_dir = os.path.join(current_dir, "explanations", "baseline")
    os.makedirs(baseline_explanations_dir, exist_ok=True)


    llm_explanations_dir = os.path.join(current_dir, "explanations", model_name)
    
    baseline_explanations = {}
    llm_explanations = {}
    for index, row in test_cases.iterrows():    
        if index not in cases_to_do:
            continue 
        print(f"Processing test case {index}")   
        # Read the JSON file into a dictionary
        with open(f'{current_dir}/{row["name_json_tree_file"]}', 'r') as file:
            json_tree = json.load(file)
        # Generate formal explanations
        formal_explentions, chosen_trace = generate_formal_explanations(json_tree=json_tree, norm=row["norm"], beliefs=row["beliefs"], goal=row["goal"], preferences=row["preferences"], action_to_explain=row["action_to_explain"])
        baseline_explanations[index] = generate_naive_baseline(formal_explentions=formal_explentions, chosen_trace=chosen_trace, norm=row["norm"], beliefs=row["beliefs"], goal=row["goal"], preferences=row["preferences"], action_to_explain=row["action_to_explain"])



        root, chosen_trace, valid_traces, valid_costs = make_decision(json_tree, norm=row["norm"], goal=row["goal"], beliefs=row["beliefs"], preferences=row["preferences"], output_dir='/')
        tree_traversal_order = preorder_traversal(root=root)

        llm_explanations[index] = generate_natural_language_explanation(
                        json_tree=json_tree,
                        naive_explanation=baseline_explanations[index],
                        formal_explanations=formal_explentions,
                        tree_traversal_order=tree_traversal_order,
                        norm=row["norm"],
                        beliefs=row["beliefs"],
                        goal=row["goal"],
                        preferences=row["preferences"],
                        action_to_explain=row["action_to_explain"],
                        chosen_trace=chosen_trace
                        )

        # Save each explanation to a separate text file
        baseline_explanation_file = os.path.join(baseline_explanations_dir, f"{index}.txt")
        with open(baseline_explanation_file, 'w') as f:
            for explanation in baseline_explanations[index]:
                f.write(explanation + '\n')


        os.makedirs(llm_explanations_dir, exist_ok=True)  

        llm_explanation_file = os.path.join(llm_explanations_dir, f"{index}.txt")

        print(f"directory ---> {llm_explanation_file}")
        
        # Loop through the dictionary and write explanations for each test case
        with open(llm_explanation_file, 'w') as f:
            for explanation in llm_explanations[index]:
                # Write the explanation for this test case
                f.write(explanation)   # Add an extra newline for separation

        
    
    return baseline_explanations, llm_explanations

In [19]:
test_cases = [
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["payShop"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [1, 2, 0]],
        "action_to_explain": "payShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {},
        "beliefs": ["staffCardAvailable", "ownCard"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "getCoffeeKitchen"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {},
        "beliefs": ["haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "getCoffeeShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["gotoAnnOffice"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "payShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["payShop"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [1, 2, 0]],
        "action_to_explain": "gotoKitchen"
    }
]

df_test_cases = pd.DataFrame(test_cases, index=[f"test_case_{i+1}" for i in range(len(test_cases))])

In [20]:
cases_to_do = ["test_case_3"] # ["test_case_1", "test_case_2", "test_case_3", "test_case_4", "test_case_5"]
baseline_explanations, llm_explanations = generate_explanations_for_inputs(df_test_cases, cases_to_do)

Processing test case test_case_3
Chosen trace: ['getCoffee', 'getShopCoffee', 'gotoShop', 'payShop', 'getCoffeeShop']
Action to explain: getCoffeeShop in trace
Ancestor names: ['getCoffee', 'getShopCoffee']
directory ---> c:\Users\bar24\OneDrive - Universiteit Utrecht\Documents\School\UU Data Sceince MSc\1st Year\Period 3\Explainable AI - INFOMXAI\Assignments\Project2\ExplainableAI-Project2\Part2\explanations\llama3\test_case_3.txt


In [21]:
print("Done!")

Done!
